# UnionFind

In [1]:
class UnionFind:
    def __init__(self, size):
        """
        Initialize the union-find data structure with a given size.

        :param size: The number of elements in the set.
        """
        self.parent = list(range(size))  # Each element is initially its own parent
        self.rank = [0] * size  # Initialize rank of each element

    def find(self, x):
        """
        Find the representative (root) of the set that element x belongs to.

        :param x: The element to find the representative for.
        :return: The representative element.
        """
        if self.parent[x] != x:
            # Path compression: Make the parent of x the root of its set
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        """
        Union the sets containing elements x and y.

        :param x: The first element.
        :param y: The second element.
        """
        root_x = self.find(x)
        root_y = self.find(y)

        if root_x != root_y:
            if self.rank[root_x] < self.rank[root_y]:
                # Attach root_x under root_y
                self.parent[root_x] = root_y
            elif self.rank[root_x] > self.rank[root_y]:
                # Attach root_y under root_x
                self.parent[root_y] = root_x
            else:
                # Arbitrarily choose one root to be the parent and increment its rank
                self.parent[root_y] = root_x
                self.rank[root_x] += 1

# Example usage
n = 5  # Number of elements
uf = UnionFind(n)  # Initialize union-find

# Perform union operations
uf.union(0, 2)
uf.union(4, 2)
uf.union(3, 1)

# Find representatives
print(uf.find(4))  # Should print the same representative as uf.find(2)

# Output the parent and rank arrays
print("Parent array:", uf.parent)
print("Rank array:", uf.rank)

0
Parent array: [0, 3, 0, 3, 0]
Rank array: [1, 0, 0, 1, 0]


# Minimal Spanning Tree(prim) Algorithm

In [5]:
import heapq

def prim_mst(graph):
    """
    Find the minimum spanning tree of a given graph using Prim's algorithm.
    
    :param graph: The input graph in the form of an adjacency list.
    :return: The edges of the minimum spanning tree.
    """
    n = len(graph)  # Number of vertices
    visited = [False] * n  # Keep track of visited vertices
    min_heap = []  # Min-heap to keep track of potential edges
    mst_edges = []  # Edges of the minimum spanning tree
    
    # Start with the first vertex (can be any vertex)
    start_vertex = 0
    visited[start_vertex] = True
    
    # Initialize the min-heap with edges from the starting vertex
    for neighbor, weight in graph[start_vertex]:
        heapq.heappush(min_heap, (weight, start_vertex, neighbor))
    
    while min_heap:
        weight, u, v = heapq.heappop(min_heap)
        
        if visited[v]:
            continue
        
        visited[v] = True
        mst_edges.append((u, v, weight))
        
        # Add new potential edges from the newly added vertex
        for neighbor, weight in graph[v]:
            if not visited[neighbor]:
                heapq.heappush(min_heap, (weight, v, neighbor))
    
    return mst_edges

# Example usage
graph = {
    0: [(1, 2), (2, 3)],
    1: [(0, 2), (2, 4), (3, 5)],
    2: [(0, 3), (1, 4), (3, 1)],
    3: [(1, 5), (2, 1)],
}

mst = prim_mst(graph)
print("Minimum Spanning Tree Edges:")
for u, v, weight in mst:
    print(f"{u} - {v} : {weight}")

Minimum Spanning Tree Edges:
0 - 1 : 2
0 - 2 : 3
2 - 3 : 1


# Shortest Path(Bellman ford) Single Source Algorithm

In [10]:
class Edge:
    def __init__(self, source, target, weight):
        self.source = source
        self.target = target
        self.weight = weight

def bellman_ford(graph, source):
    """
    Find the shortest paths from a given source vertex to all other vertices
    in the graph using the Bellman-Ford algorithm.
    
    :param graph: The input graph in the form of a list of Edge objects.
    :param source: The source vertex.
    :return: A list of distances from the source to all other vertices.
    """
    n = len(graph)  # Number of vertices
    distances = [float('inf')] * n  # Initialize distances to infinity
    distances[source] = 0  # Distance to source is 0
    
    # Relax edges (n-1) times to ensure optimal shortest paths
    for _ in range(n - 1):
        for edge in graph:
            if distances[edge.source] + edge.weight < distances[edge.target]:
                distances[edge.target] = distances[edge.source] + edge.weight
    
    # Check for negative cycles
    for edge in graph:
        if distances[edge.source] + edge.weight < distances[edge.target]:
            raise ValueError("Graph contains a negative cycle")
    
    return distances

# Example usage
edges = [
    Edge(0, 1, 5),
    Edge(0, 2, 3),
    Edge(1, 3, 6),
    Edge(2, 1, -2),
    Edge(2, 3, 2),
    Edge(3, 4, 3),
    Edge(4, 2, 7),
]

source_vertex = 0
shortest_distances = bellman_ford(edges, source_vertex)
print("Shortest distances from source vertex", source_vertex)
for i, distance in enumerate(shortest_distances):
    print(f"To vertex {i}: {distance}")

Shortest distances from source vertex 0
To vertex 0: 0
To vertex 1: 1
To vertex 2: 3
To vertex 3: 5
To vertex 4: 8
To vertex 5: inf
To vertex 6: inf


# Shortest Path(Dijkstra) Single Source Algorithm

In [17]:
import heapq

class Edge:
    def __init__(self, target, weight):
        self.target = target
        self.weight = weight

def dijkstra(graph, source):
    """
    Find the shortest paths from a given source vertex to all other vertices
    in the graph using Dijkstra's algorithm.
    
    :param graph: The input graph in the form of a list of lists of Edge objects.
    :param source: The source vertex.
    :return: A list of distances from the source to all other vertices.
    """
    n = len(graph)  # Number of vertices
    distances = [float('inf')] * n  # Initialize distances to infinity
    distances[source] = 0  # Distance to source is 0
    
    # Use a min-heap to track vertices and their tentative distances
    min_heap = [(0, source)]
    
    while min_heap:
        dist, current_vertex = heapq.heappop(min_heap)
        
        # Skip vertices that have already been processed
        if dist > distances[current_vertex]:
            continue
        
        # Explore neighbors of the current vertex
        for edge in graph[current_vertex]:
            new_distance = distances[current_vertex] + edge.weight
            
            if new_distance < distances[edge.target]:
                distances[edge.target] = new_distance
                heapq.heappush(min_heap, (new_distance, edge.target))
    
    return distances

# Example usage
graph = [
    [Edge(1, 5), Edge(2, 3)],
    [Edge(3, 6)],
    [Edge(1, -2), Edge(3, 2)],
    [Edge(4, 3)],
    [Edge(2, 7)]
]

source_vertex = 0
shortest_distances = dijkstra(graph, source_vertex)
print("Shortest distances from source vertex", source_vertex)
for i, distance in enumerate(shortest_distances):
    print(f"To vertex {i}: {distance}")

Shortest distances from source vertex 0
To vertex 0: 0
To vertex 1: 1
To vertex 2: 3
To vertex 3: 5
To vertex 4: 8


In [ ]:
import heapq

class Edge:
    def __init__(self, target, weight):
        self.target = target
        self.weight = weight
        
def dijkstra(graph, source):
    n = len(graph)
    distance = [float('inf') * n]
    distance[source] = 0
    
    min_heap = [(0, source)]
    
    while min_heap:
        dist, current_vertex = heapq.heappop(min_heap)
        
        if dist > distance[current_vertex]:
            continue
            
        for edge in graph[current_vertex]:
            new_dist = distance[current_vertex] + edge.weight
            
            if new_dist < distance[edge.target]:
                distance[edge.target] = new_dist
                heapq.heappush(minheap)



graph = [
    [Edge(1, 5), Edge(2, 3)],
    [Edge(3, 6)],
    [Edge(1, -2), Edge(3, 2)],
    [Edge(4, 3)],
    [Edge(2, 7)]
]

source_vertex = 0
shortest_distances = dijkstra(graph, source_vertex)
print("Shortest distances from source vertex", source_vertex)
for i, distance in enumerate(shortest_distances):
    print(f"To vertex {i}: {distance}")

# Shortest Path(floyd Warshall) all pair algorithm

In [74]:
def floyd_warshall(graph):
    """
    Find the shortest paths between all pairs of vertices in the graph
    using the Floyd-Warshall algorithm.
    
    :param graph: The input graph in the form of an adjacency matrix.
    :return: A matrix of shortest distances between all pairs of vertices.
    """
    n = len(graph)  # Number of vertices
    
    # Initialize the distance matrix with the input graph
    distances = [[float('inf')] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i == j:
                distances[i][j] = 0
            elif graph[i][j] != 0:
                distances[i][j] = graph[i][j]
    
    # Compute shortest paths between all pairs of vertices
    for k in range(n):
        for i in range(n):
            for j in range(n):
                distances[i][j] = min(distances[i][j], distances[i][k] + distances[k][j])
    
    return distances

# Example usage
inf = float('inf')
graph = [
    [0, 3, inf, 7],
    [8, 0, 2, inf],
    [5, inf, 0, 1],
    [2, inf, inf, 0]
]

shortest_distances = floyd_warshall(graph)
print("Shortest distances between all pairs of vertices:")
for row in shortest_distances:
    print(row)

Shortest distances between all pairs of vertices:
[0, 3, 5, 6]
[5, 0, 2, 3]
[3, 6, 0, 1]
[2, 5, 7, 0]
